# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.12,-134.97,70.72,71,0,21.99,PF,1597020579
1,1,Albany,42.60,-73.97,81.00,63,0,4.45,US,1597020579
2,2,Punta Arenas,-53.15,-70.92,33.80,80,100,16.11,CL,1597020579
3,3,Deputatsky,69.30,139.90,37.69,95,100,6.33,RU,1597020579
4,4,Airai,-8.93,125.41,74.05,56,3,0.83,TL,1597020579
...,...,...,...,...,...,...,...,...,...,...
564,564,Mandalgovi,45.76,106.27,59.11,57,96,14.79,MN,1597020759
565,565,Mingoyo,-10.10,39.63,66.20,93,20,6.93,TZ,1597020759
566,566,Mugur-Aksy,50.35,90.50,54.82,63,100,1.30,RU,1597020759
567,567,Itaituba,-4.28,-55.98,77.14,49,0,0.20,BR,1597020759


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store 'Lat' and 'Lng' into  locations, and humidity column into humidity 
locations = city_data_df[["Lat", "Lng"]].astype(float)
humidity = city_data_df["Humidity"]

# Create a humidity Heatmap
fig = gmaps.figure(center = (20,0), zoom_level = 2)


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
perfect_cities_df = (city_data_df.loc[(city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & \
                                      (city_data_df["Wind Speed"] < 10) & \
                                      (city_data_df["Cloudiness"] == 0) & \
                                      (city_data_df["Humidity"] > 45) & (city_data_df["Humidity"] < 65)]).dropna()
perfect_cities_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,Santa Marinella,42.03,11.85,75.00,57,0,0.81,IT,1597020596
124,124,Tataouine,32.93,10.45,75.20,64,0,2.24,TN,1597020616
133,133,Concordia,-31.39,-58.02,72.00,60,0,4.00,AR,1597020617
134,134,Yangi Marg`ilon,40.43,71.72,73.40,53,0,6.93,UZ,1597020617
212,212,Santa Fe,-31.63,-60.70,73.00,59,0,1.99,AR,1597020620
330,330,Kogon,39.72,64.55,71.60,64,0,4.70,UZ,1597020672
517,517,Salto,-31.38,-57.97,72.00,60,0,4.00,UY,1597020744
567,567,Itaituba,-4.28,-55.98,77.14,49,0,0.20,BR,1597020759


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# finding the first hotel for each city located within 5000 meters of the coordinates, (lat,lng)

import json

hotel_list = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
}


# use iterrows to iterate through pandas dataframe
for i, row in perfect_cities_df.iterrows():
    
    # get the latitude and longitude values
    lat = row["Lat"]
    lng = row["Lng"]
    City = row["City"]
    
    # add location to params dict
    params["location"] = (f"{lat},{lng}")
    
    # assemble url and make API request
    # print(f"Retrieving Results for City {City}.")
    response = requests.get(base_url, params=params).json()
    
    # Printing the response so we can visualy locate the first hotel
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response["results"]
    x = (results[0]["name"])
    print(f"Retrieving Results for City {City}.")
    print(f"The first hotel is {x}.")
    hotel_list.append(x)


Retrieving Results for City Santa Marinella.
The first hotel is VILLA GREGORACI - b & b, guest houses, Santa Marinella.
Retrieving Results for City Tataouine.
The first hotel is Sangho Privilege Tataouine.
Retrieving Results for City Concordia.
The first hotel is Palmar Hotel Casino.
Retrieving Results for City Yangi Marg`ilon.
The first hotel is Ikathouse.
Retrieving Results for City Santa Fe.
The first hotel is Conquistador Hotel.
Retrieving Results for City Kogon.
The first hotel is Hostel "New Bukhara".
Retrieving Results for City Salto.
The first hotel is Gran Hotel Uruguay.
Retrieving Results for City Itaituba.
The first hotel is Hotel Campos.


In [9]:
perfect_cities_df["Hotel Name"] = hotel_list
perfect_cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,54,Santa Marinella,42.03,11.85,75.00,57,0,0.81,IT,1597020596,"VILLA GREGORACI - b & b, guest houses, Santa M..."
124,124,Tataouine,32.93,10.45,75.20,64,0,2.24,TN,1597020616,Sangho Privilege Tataouine
133,133,Concordia,-31.39,-58.02,72.00,60,0,4.00,AR,1597020617,Palmar Hotel Casino
134,134,Yangi Marg`ilon,40.43,71.72,73.40,53,0,6.93,UZ,1597020617,Ikathouse
212,212,Santa Fe,-31.63,-60.70,73.00,59,0,1.99,AR,1597020620,Conquistador Hotel
330,330,Kogon,39.72,64.55,71.60,64,0,4.70,UZ,1597020672,"Hostel ""New Bukhara"""
517,517,Salto,-31.38,-57.97,72.00,60,0,4.00,UY,1597020744,Gran Hotel Uruguay
567,567,Itaituba,-4.28,-55.98,77.14,49,0,0.20,BR,1597020759,Hotel Campos


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_cities_df.iterrows()]
locations = perfect_cities_df[["Lat", "Lng"]]

In [11]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info )
# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))